# File Description

This notebook contains the most contains the latest version of the time dependent BN modeling for Tarawa, Kiribati.

# File Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import os
import json
from datetime import datetime, time, timedelta
import pysmile
import pysmile_license
import sys
import json
sys.path.append('/src/python_classes')
import rpy2
# os.environ['R_HOME'] = 'C:\ProgramData\Anaconda3\Lib\R'
# %load_ext rpy2.ipython
!jupyter nbextension enable --py --sys-prefix ipyleaflet
from ipywidgets import interact, interactive, fixed, interact_manual
from ipyleaflet import *
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
import geojson
import folium
from folium.plugins import FloatImage as FloatImage
from colormap import rgb2hex
import rpy2
os.environ['R_HOME'] = '/lib/R'
%load_ext rpy2.ipython

from BNModel import BNModel

from preprocessing_all_points import *
from preprocessing_points_spatially_temporally import *
from compile_model_t import *

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


In [4]:
# ### set location of file storage
# folder = 'BN_antonio_data'
# try:
#     os.makedirs(folder)
# except FileExistsError:
#     pass

# Data Prep

## Preprocessing

In [125]:
# Import and preprocess data
df_lagoon_profiles,df_ocean_profiles,inundation_dict,winds_dict,waves_dict,tide_dict,sla_dict,time_dict = \
    loading_tarawa_data()

In [139]:
sla_dict.keys()

dict_keys(['MSL'])

In [140]:
df_ocean_profiles

,shore_long,shore_lat,reef_long,reef_lat,reef_width,reef_depth,forereef_slope,m,shore_dir
0,172.92001,1.359241,172.90162,1.380442,1815.70350,2.532653,0.018279,-1.152871,319.061643
1,172.91848,1.356421,172.89642,1.357271,389.26039,0.762624,0.004784,-0.038531,272.206588
2,172.92019,1.353139,172.91862,1.342910,639.97829,1.268571,0.088677,6.514968,188.726378
3,172.92488,1.353647,172.92461,1.342294,673.60616,1.124577,0.075739,42.051111,181.362270
4,172.92935,1.353986,172.92990,1.342910,685.94954,1.055773,0.088621,-20.138000,177.157178
...,...,...,...,...,...,...,...,...,...
177,172.91907,1.357814,172.89827,1.369253,521.44384,1.481050,0.005864,-0.549976,298.809736
178,172.91862,1.354158,172.90065,1.346258,1516.82890,1.841662,0.027996,0.439622,246.268673
179,172.95727,1.636152,172.95393,1.645588,618.70990,1.091268,0.060486,-2.824940,340.506556
180,173.04626,1.356990,173.04952,1.348703,525.15634,1.362362,0.086555,-2.542086,158.526428


## Create Variable Dictionary

In [151]:
def initialise_model_dictionaries():
    #### Don't include spaces in bin names. if no discretisation, just leave out that key
    lagoon_model_dict = {
        'variables':{
            'wind_u':{
                'label':'Wind u vector (m/s)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'wind_v':{
                'label':r'Wind v vector (m/s)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'uniform',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'Hs_offshore':{
                'label':'Offshore wave height (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','HighMid','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'Tm_offshore':{
                'label':'Offshore wave period (s)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'Dir_offshore':{
                'label':r'Offshore wave direction (degrees)',
                'discretisation':{
                    'n_bins':8,
                    'strategy':'kmeans',
                    'bin_names':['NNE','ENE','ESE','SSE','SSW','WSW','WNW','NNW']
                },
                'child_nodes':['WL_wave_comp']
            },
            'WL_wave_comp':{
                'label':'Water level from wave component (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':[]
            }
        },
        'training_frac':0.8,
        'bootstrap_reps':1
    }
    ocean_model_dict = {
       'variables':{
           'Tm_offshore':{
                'label':'Wave period offshore (?)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'Hs_offshore':{
                'label':'Wave height offshore (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'Dir_offshore':{
                'label':'Wave direction offshore (degrees)',
                'discretisation':{
                    'n_bins':8,
                    'strategy':'kmeans',
                    'bin_names':['NNE','ENE','ESE','SSE','SSW','WSW','WNW','NNW']
                },
                'child_nodes':['WL_wave_comp']
            },
            'WL_wave_comp':{
                'label':'Water level from wave component (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':[]
            },
            'reef_width':{
                'label':'Reef width (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'reef_depth':{
                'label':'Reef depth (m)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'forereef_slope':{
                'label':'Fore reef slope (degrees)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['VeryLow','Low','Mid','High','VeryHigh']
                },
                'child_nodes':['WL_wave_comp']
            },
            'shore_dir':{
                'label':'Shoreline direction (degrees)',
                'discretisation':{
                    'n_bins':3,
                    'strategy':'kmeans',
                    'bin_names':['NE','S','NW']
                },
                'child_nodes':['WL_wave_comp']
            }
       },
        'training_frac':0.8,
        'bootstrap_reps':1
    }
    return(lagoon_model_dict,ocean_model_dict)

In [7]:
def bin_locator(value,bin_edges):
    '''
    function used for determining the index of the appropriate bin for a numerical value.
    '''
    i=0
    for edge_1,edge_2 in zip(bin_edges[:-1],bin_edges[1:]):
        if (value>edge_1)&(value<=edge_2):
            loc_bin = i
        else:
            i+=1
            continue

    if value<=bin_edges[0]:
        loc_bin = 0

    if value>=bin_edges[-1]:
        loc_bin = len(bin_edges)-2

    return(loc_bin)

def model_location(model_dict,location_details,evidence_dict,variable_list):
    
    '''
    function for adding the location information for one side model to the evidence dictionary
    '''
    
    for variable in variable_list:

        bin_edges = model_dict['variables'][variable]['bin_edges'][0]
        value = location_details[variable]

        var_bin = bin_locator(value,bin_edges)
        
        evidence_array = [0]*(len(bin_edges)-1)
        evidence_array[var_bin] = 1
        
        evidence_dict.update({
            variable:evidence_array
        })

    # Add evidence to model dict
    model_location_dict = BNModel().add_evidence_to_dict(model_dict,evidence_dict)
    
    # Set evidence and get beliefs
    model_location_dict = BNModel().update_evidence(model_location_dict)
    
    return(model_location_dict)

def location_probabilities(evidence_dict,model_dict,variable_list,df_profiles):
    '''
    
    Function for setting evidence and determing probabilties for twl at each point around the island based 
    on the reef characteristics at each location
    
    '''
    
    location_probabilities_dict = {}
    
    for index,row in df_profiles.iterrows():

        model_location_dict = model_location(model_dict,row,evidence_dict,variable_list)
        location_probabilities = model_location_dict['variables']['TWL']['resulting_probs'][0]

        df_location_probabilities = pd.DataFrame.from_dict(location_probabilities,orient='index')
        
        largest_cat = df_location_probabilities.idxmax()[0]

        location_probabilities_dict.update({
            (row.reef_long,row.reef_lat):\
                model_dict['variables']['TWL']['discretisation']['bin_names'].index(largest_cat)
        })
        
    return(location_probabilities_dict)

## Bootstrapping the data

# One network per time step

In [228]:

model_dicts_through_time_dict = {}

time_min = inundation_dict['Time'][0]
time_step = timedelta(hours=24)
max_steps = 7

times_for_model = []

for step in np.arange(0,max_steps,1):
    time_at_step = time_min+time_step*step
    
    times_for_model.append(time_at_step)
    
times_for_model = times_for_model[:10]
    
for time_min,time_max in zip(times_for_model[:-2],times_for_model[2:]):
    lower_idx = np.abs([x-time_min for x in inundation_dict['Time']]).argmin()
    upper_idx = np.abs([x-time_max for x in inundation_dict['Time']]).argmin()
       
    inundation_time_slice_dict = {var:inundation_dict[var][lower_idx:upper_idx,:] for var in ['TWL','Tide']}
    inundation_time_slice_dict['Time'] = inundation_dict['Time'][lower_idx:upper_idx]
    inundation_time_slice_dict['Ptos'] = inundation_dict['Ptos']
    winds_time_slice_dict = {
        'wind_u':winds_dict['wind_u'][lower_idx:upper_idx],'wind_v':winds_dict['wind_v'][lower_idx:upper_idx]}
    waves_time_slice_dict = {var:waves_dict[var][lower_idx:upper_idx,:] for var in ['Diro','Hso','Tmo','Tpo']}
    waves_time_slice_dict['Timeo'] = waves_dict['Timeo'][lower_idx:upper_idx]
    
    tide_time_slice_dict = {'Tide':tide_dict['Tide'][lower_idx:upper_idx]}
    sla_time_slice_dict = {'MSL':sla_dict['MSL'][lower_idx:upper_idx]}
    time_slice_dict = {'time':time_dict['time'][lower_idx:upper_idx]}
    
#     if time_min==times_for_model[0]:
#         lagoon_model_dict,ocean_model_dict = initialise_model_dictionaries()
#         predicted_lag_variables = []
#     else:
#         predicted_lag_variables = ['TWL_t_1']
#         for var in predicted_lag_variables:
#             df_ocean_profiles[var] = 0#[item for key,item in ocean_probabilities_dict.items()]
            
#         ocean_model_dict['variables'].update({
#                 'TWL_t_1':{
#                 'label':'Total water level bin at t-1 (1-5?)',
#                 'discretisation':{
#                     'n_bins':5,
#                     'strategy':'kmeans',
#                     'bin_names':['1','2','3','4','5']
#                 },
#                 'child_nodes':['TWL']
#             }
#         })

    df_ocean,df_lagoon = \
        preprocessing_points_spatially_temporally(df_lagoon_profiles,
                                                  df_ocean_profiles,
                                                  inundation_time_slice_dict,
                                                  winds_time_slice_dict,
                                                  waves_time_slice_dict,
                                                  tide_time_slice_dict,
                                                  sla_time_slice_dict,
                                                  time_slice_dict,
                                                  predicted_lag_variables)
    
    # Define which variables are to be time lagged
    ocean_time_delay_vars = ['time','WL_wave_comp']
    lagoon_time_delay_vars = ['time','WL_wave_comp','wind_u','wind_v','Dir_offshore','Hs_offshore','Tm_offshore']

    # Because the lag is going to depend on the location, define the list of variables which are used to group by loc
    location_vars_list = ['lat','long']

    # Add the time lag variables
    df_ocean_with_lag = add_time_lag(df_ocean,location_vars_list,ocean_time_delay_vars)
    df_lagoon_with_lag = add_time_lag(df_lagoon,location_vars_list,lagoon_time_delay_vars)

    # Create dictionary of variables
    lagoon_data_dict = {column:np.array(df_lagoon_with_lag[column]) for column in df_lagoon_with_lag.columns}
    ocean_data_dict = {column:np.array(df_ocean_with_lag[column]) for column in df_ocean_with_lag.columns}

    # Remove the variables that I don't want to include in the model
    lagoon_data_dict = {key:item for key,item in lagoon_data_dict.items() if key not in ['lat','long','time','TWL','Tide','MSL']}
    ocean_data_dict = {key:item for key,item in ocean_data_dict.items() if key not in ['lat','long','time','wind_u','wind_v','TWL','Tide','MSL']}

    # Duplicate the variables to make T-1 in the model dictionary
    for var in ocean_time_delay_vars:
        if var!='time':
            ocean_model_dict['variables'][var+'_t_1'] = ocean_model_dict['variables'][var].copy()
    for var in lagoon_time_delay_vars:
        if var!='time':
            lagoon_model_dict['variables'][var+'_t_1'] = lagoon_model_dict['variables'][var].copy()
    
    lagoon_model_dict,ocean_model_dict = create_BN_time_t(lagoon_model_dict,
                                                          lagoon_data_dict,
                                                          df_ocean_with_lag,
                                                          ocean_model_dict,
                                                          df_ocean_with_lag,
                                                          ocean_data_dict)    
    
    
    # Create an empty evidence dict
    evidence_dict = {}
    
    # Add evidence to model dict
    ocean_model_dict = BNModel().add_evidence_to_dict(ocean_model_dict,evidence_dict)
    lagoon_model_dict = BNModel().add_evidence_to_dict(lagoon_model_dict,evidence_dict)

    # Set evidence and get beliefs
    ocean_model_dict = BNModel().update_evidence(ocean_model_dict)
    lagoon_model_dict = BNModel().update_evidence(lagoon_model_dict)
    
    # Get the resulting TWL probabilities (right now for a generic location)
    resulting_probs_ocean = ocean_model_dict['variables']['TWL']['resulting_probs']
    resulting_probs_lagoon = lagoon_model_dict['variables']['TWL']['resulting_probs']
    
    model_dicts_through_time_dict.update({
        time_dict['time'][lower_idx]:{
            'lagoon':lagoon_model_dict,
            'ocean':ocean_model_dict
        }
    })

IndexError: arrays used as indices must be of integer (or boolean) type

In [230]:
# Notes from above: I have decided that the lag won't include prediction. I have now added in the lag variable into the lagoon and the ocean dataframe and the dictionary
# Now I just need to troubleshoot the consequences

{'wind_u': array([1.9594574 , 1.97935486, 2.06712341, ..., 4.75270081, 5.67976379,
        5.59844971]),
 'wind_v': array([-3.86637878, -3.70275879, -3.37741089, ..., -4.07032776,
        -4.42909241, -5.48405457]),
 'WL_wave_comp': array([0.01463386, 0.01421414, 0.01381309, ..., 0.03235189, 0.03278912,
        0.03537253]),
 'Hs_offshore': array([1.46467876, 1.45756996, 1.44986725, ..., 1.40239811, 1.41429865,
        1.43457103]),
 'Tm_offshore': array([7.75293827, 7.78184128, 7.90084076, ..., 4.7946887 , 4.75499535,
        4.58586025]),
 'Dir_offshore': array([210.91932678, 210.65341187, 210.34597778, ...,  51.61738586,
         51.31286621,  49.61532593]),
 'WL_wave_comp_t_1': array([0.01495346, 0.01463386, 0.01421414, ..., 0.03087198, 0.03235189,
        0.03278912]),
 'wind_u_t_1': array([1.93714905, 1.9594574 , 1.97935486, ..., 4.79971313, 4.75270081,
        5.67976379]),
 'wind_v_t_1': array([-3.85470581, -3.86637878, -3.70275879, ..., -4.13745117,
        -4.07032776, -4.429

In [155]:
list(group)

['TWL',
 'Tide',
 'MSL',
 'wind_u',
 'wind_v',
 'time',
 'WL_wave_comp',
 'Hs_offshore',
 'Tm_offshore',
 'Dir_offshore',
 'reef_width',
 'reef_depth',
 'forereef_slope',
 'long',
 'lat',
 'shore_dir']

In [163]:
list(df_lagoon)

['TWL',
 'Tide',
 'MSL',
 'wind_u',
 'wind_v',
 'time',
 'WL_wave_comp',
 'Hs_offshore',
 'Tm_offshore',
 'Dir_offshore',
 'long',
 'lat']

In [185]:
def add_time_lag(df,location_vars_list,time_delay_vars):
    previous_time_dict = {}

    for loc_vars,group in df.groupby(location_vars_list):
        df_orig = group.reset_index(drop=True)
        df_delay = group[time_delay_vars].iloc[:-1].reset_index(drop=True)
        df_delay.index = [x+1 for x in list(df_delay.index)]
        df_delay.drop('time',axis=1,inplace=True)
        group = df_orig.join(df_delay,rsuffix='_t_1',how='inner')

        previous_time_dict.update({
            loc_vars:group
        })

    df = pd.concat(previous_time_dict)
    
    return(df)

In [223]:
# Define which variables are to be time lagged
ocean_time_delay_vars = ['time','WL_wave_comp']
lagoon_time_delay_vars = ['time','WL_wave_comp','wind_u','wind_v','Dir_offshore','Hs_offshore','Tm_offshore']

# Because the lag is going to depend on the location, define the list of variables which are used to group by loc
location_vars_list = ['lat','long']

# Add the time lag variables
df_ocean_with_lag = add_time_lag(df_ocean,location_vars_list,ocean_time_delay_vars)
df_lagoon_with_lag = add_time_lag(df_lagoon,location_vars_list,lagoon_time_delay_vars)

# Create dictionary of variables
lagoon_data_dict = {column:np.array(df_lagoon_with_lag[column]) for column in df_lagoon_with_lag.columns}
ocean_data_dict = {column:np.array(df_ocean_with_lag[column]) for column in df_ocean_with_lag.columns}

# # Remove the variables that I don't want to include in the model
# lagoon_data_dict = {key:item for key,item in lagoon_data_dict.items() if key not in ['lat','long','time','TWL','Tide','MSL']}
# ocean_data_dict = {key:item for key,item in ocean_data_dict.items() if key not in ['lat','long','time','wind_u','wind_v','TWL','Tide','MSL']}

# # Duplicate the variables to make T-1 in the model dictionary
# for var in ocean_time_delay_vars:
#     if var!='time':
#         ocean_model_dict['variables'][var+'_t_1'] = ocean_model_dict['variables'][var].copy()
# for var in lagoon_time_delay_vars:
#     if var!='time':
#         lagoon_model_dict['variables'][var+'_t_1'] = lagoon_model_dict['variables'][var].copy()

In [215]:
df_lagoon_with_lag

TWL      Tide       MSL    wind_u    wind_v  \
1.329805 172.98940 1   0.041527 -0.018809  0.045702  1.959457 -3.866379   
                   2  -0.076081 -0.136272  0.045977  1.979355 -3.702759   
                   3  -0.146709 -0.206774  0.046251  2.067123 -3.377411   
                   4  -0.154925 -0.214885  0.046526  2.052292 -3.127548   
                   5  -0.099177 -0.160328  0.046800  2.737732 -2.956863   
...                         ...       ...       ...       ...       ...   
1.608513 172.95703 43 -0.161954 -0.249675  0.060320  5.745117 -3.376038   
                   44 -0.038642 -0.130271  0.060757  4.799713 -4.137451   
                   45  0.117090  0.023545  0.061194  4.752701 -4.070328   
                   46  0.272904  0.178484  0.061631  5.679764 -4.429092   
                   47  0.397587  0.300146  0.062068  5.598450 -5.484055   

                                            time  WL_wave_comp  Hs_offshore  \
1.329805 172.98940 1  1993-01-01 00:59:59.999997      0.014634     1.464679   
                   2  1993-01-01 02:00:00.000003      0.014214     1.457570   
                   3  1993-01-01 03:00:00.000000      0.013813     1.449867   
                   4  1993-01-01 03:59:59.999997      0.013434     1.444235   
                   5  1993-01-01 05:00:00.000003      0.014351     1.442475   
...                                          ...           ...          ...   
1.608513 172.95703 43 1993-01-02 18:59:59.999997      0.027402     1.381990   
                   44 1993-01-02 20:00:00.000003      0.030872     1.391962   
                   45 1993-01-02 21:00:00.000000      0.032352     1.402398   
                   46 1993-01-02 21:59:59.999997      0.032789     1.414299   
                   47 1993-01-02 23:00:00.000003      0.035373     1.434571   

                       Tm_offshore  Dir_offshore       long       lat  \
1.329805 172.98940 1      7.752938    210.919327  172.98940  1.329805   
                   2      7.781841    210.653412  172.98940  1.329805   
                   3      7.900841    210.345978  172.98940  1.329805   
                   4      8.029340    209.973969  172.98940  1.329805   
                   5      7.988736    209.568527  172.98940  1.329805   
...                            ...           ...        ...       ...   
1.608513 172.95703 43     4.618346     50.048782  172.95703  1.608513   
                   44     4.698277     50.922104  172.95703  1.608513   
                   45     4.794689     51.617386  172.95703  1.608513   
                   46     4.754995     51.312866  172.95703  1.608513   
                   47     4.585860     49.615326  172.95703  1.608513   

                       WL_wave_comp_t_1  wind_u_t_1  wind_v_t_1  \
1.329805 172.98940 1           0.014953    1.937149   -3.854706   
                   2           0.014634    1.959457   -3.866379   
                   3           0.014214    1.979355   -3.702759   
                   4           0.013813    2.067123   -3.377411   
                   5           0.013434    2.052292   -3.127548   
...                                 ...         ...         ...   
1.608513 172.95703 43          0.024418    6.494019   -3.081268   
                   44          0.027402    5.745117   -3.376038   
                   45          0.030872    4.799713   -4.137451   
                   46          0.032352    4.752701   -4.070328   
                   47          0.032789    5.679764   -4.429092   

                       Dir_offshore_t_1  Hs_offshore_t_1  Tm_offshore_t_1  
1.329805 172.98940 1         211.209900         1.469232         7.602891  
                   2         210.919327         1.464679         7.752938  
                   3         210.653412         1.457570         7.781841  
                   4         210.345978         1.449867         7.900841  
                   5         209.973969         1.444235         8.029340  
...                                 ... 

[numpy.datetime64('1993-01-01T00:59:59.999997000'),
 numpy.datetime64('1993-01-01T02:00:00.000003000'),
 numpy.datetime64('1993-01-01T03:00:00.000000000'),
 numpy.datetime64('1993-01-01T03:59:59.999997000'),
 numpy.datetime64('1993-01-01T05:00:00.000003000'),
 numpy.datetime64('1993-01-01T06:00:00.000000000'),
 numpy.datetime64('1993-01-01T06:59:59.999997000'),
 numpy.datetime64('1993-01-01T08:00:00.000003000'),
 numpy.datetime64('1993-01-01T09:00:00.000000000'),
 numpy.datetime64('1993-01-01T09:59:59.999997000'),
 numpy.datetime64('1993-01-01T11:00:00.000003000'),
 numpy.datetime64('1993-01-01T12:00:00.000000000'),
 numpy.datetime64('1993-01-01T12:59:59.999997000'),
 numpy.datetime64('1993-01-01T14:00:00.000003000'),
 numpy.datetime64('1993-01-01T15:00:00.000000000'),
 numpy.datetime64('1993-01-01T15:59:59.999997000'),
 numpy.datetime64('1993-01-01T17:00:00.000003000'),
 numpy.datetime64('1993-01-01T18:00:00.000000000'),
 numpy.datetime64('1993-01-01T18:59:59.999997000'),
 numpy.datet

In [203]:
# Duplicate the variables to make T-1 in the model dictionary
for var in ocean_time_delay_vars:
    if var!='time':
        ocean_model_dict['variables'][var+'_t_1'] = ocean_model_dict['variables'][var].copy()
for var in lagoon_time_delay_vars:
    if var!='time':
        lagoon_model_dict['variables'][var+'_t_1'] = lagoon_model_dict['variables'][var].copy()

In [204]:
lagoon_model_dict['variables'].keys()

dict_keys(['wind_u', 'wind_v', 'Hs_offshore', 'Tm_offshore', 'Dir_offshore', 'WL_wave_comp', 'WL_wave_comp_t_1', 'wind_u_t_1', 'wind_v_t_1', 'Dir_offshore_t_1', 'Hs_offshore_t_1', 'Tm_offshore_t_1'])

In [70]:

variable_list = ['reef_width','reef_depth','forereef_slope','shore_dir']

# Create an empty location probability dictionary if this is the first timestep
if time_min==times_for_model[0]:
    location_probabilities_dict = {}

# Loop over each location, and create dictionary of each of the reef characteristics
for index,row in df_ocean_profiles.iterrows():

    evidence_dict = {}

    for variable in variable_list:

        bin_edges = ocean_model_dict['variables'][variable]['bin_edges'][0]
        value = row[variable]

        var_bin = bin_locator(value,bin_edges)
        
        evidence_array = [0]*(len(bin_edges)-1)
        evidence_array[var_bin] = 1
        
        evidence_dict.update({
            variable:evidence_array
        })
        
    # Now add previous TWL probabilities to dict
    if time_min!=times_for_model[0]:
        # have to add a new node for TWL t-1
        predicted_lag_variables = ['TWL_t_1']
        for var in predicted_lag_variables:
            df_ocean_profiles[var] = 0#[item for key,item in ocean_probabilities_dict.items()]
            
        ocean_model_dict['variables'].update({
                'TWL_t_1':{
                'label':'Total water level bin at t-1 (1-5?)',
                'discretisation':{
                    'n_bins':5,
                    'strategy':'kmeans',
                    'bin_names':['1','2','3','4','5']
                },
                'child_nodes':['TWL']
            }
        })
        # Add twl at t-1 to evidence dict
        evidence_dict.update({
            'TWL_t_1':location_probabilities_dict[(row.reef_long,row.reef_lat)]

    # Add evidence to model dict
    ocean_model_location_dict = BNModel().add_evidence_to_dict(ocean_model_dict,evidence_dict)
    
    location_probabilities = ocean_model_location_dict['variables']['TWL']['resulting_probs'][0]

    location_probabilities_dict.update({
        (row.reef_long,row.reef_lat):[x for y,x in location_probabilities.items()]
    })


{'reef_width': [0, 0, 0, 0, 1], 'reef_depth': [0, 0, 0, 0, 1], 'forereef_slope': [1, 0, 0, 0, 0], 'shore_dir': [0, 0, 1]}
{'reef_width': [0, 1, 0, 0, 0], 'reef_depth': [1, 0, 0, 0, 0], 'forereef_slope': [1, 0, 0, 0, 0], 'shore_dir': [0, 0, 1]}
{'reef_width': [0, 0, 1, 0, 0], 'reef_depth': [0, 0, 1, 0, 0], 'forereef_slope': [0, 1, 0, 0, 0], 'shore_dir': [0, 1, 0]}
{'reef_width': [0, 0, 1, 0, 0], 'reef_depth': [0, 1, 0, 0, 0], 'forereef_slope': [0, 1, 0, 0, 0], 'shore_dir': [0, 1, 0]}
{'reef_width': [0, 0, 1, 0, 0], 'reef_depth': [0, 1, 0, 0, 0], 'forereef_slope': [0, 1, 0, 0, 0], 'shore_dir': [0, 1, 0]}
{'reef_width': [0, 0, 1, 0, 0], 'reef_depth': [0, 1, 0, 0, 0], 'forereef_slope': [0, 1, 0, 0, 0], 'shore_dir': [0, 1, 0]}
{'reef_width': [0, 0, 1, 0, 0], 'reef_depth': [0, 1, 0, 0, 0], 'forereef_slope': [0, 0, 1, 0, 0], 'shore_dir': [0, 1, 0]}
{'reef_width': [0, 0, 1, 0, 0], 'reef_depth': [0, 0, 1, 0, 0], 'forereef_slope': [0, 1, 0, 0, 0], 'shore_dir': [0, 1, 0]}
{'reef_width': [0, 1, 0,

[0.0,
 0.06998327275320608,
 0.1909362259983346,
 0.25468698404782525,
 0.2544471175922037,
 0.18152933108769803,
 0.04841706852073222]

In [64]:
ocean_model_dict['variables']['TWL']['discretisation']

{'n_bins': 7,
 'strategy': 'binned',
 'bin_names': ['VeryLow',
  'Low',
  'LowMid',
  'Mid',
  'MidHigh',
  'High',
  'VeryHigh'],
 'bin_edges': array([-1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5])}

In [55]:
location_probabilities_dict

{(172.90162, 1.3804422): 3,
 (172.89642, 1.3572711): 3,
 (172.91862, 1.3429104): 3,
 (172.92461, 1.3422936): 3,
 (172.9299, 1.3429104): 3,
 (172.93386, 1.343439): 3,
 (172.93633, 1.3425579): 3,
 (172.93906, 1.3415007): 3,
 (172.9425, 1.3401792): 3,
 (172.94681, 1.3384171): 3,
 (172.95016, 1.3369194): 3,
 (172.95302, 1.3344084): 3,
 (172.95606, 1.3321618): 3,
 (172.96069, 1.3299592): 3,
 (172.96461, 1.3270518): 3,
 (172.9691, 1.3238361): 3,
 (172.97192, 1.3219419): 3,
 (172.97606, 1.3214133): 3,
 (172.97972, 1.3210168): 3,
 (172.9847, 1.3209287): 3,
 (172.98923, 1.3214573): 3,
 (172.99404, 1.3225145): 3,
 (172.99813, 1.322911): 3,
 (173.00302, 1.3241444): 3,
 (173.00703, 1.32551): 3,
 (173.00884, 1.3257743): 3,
 (173.01161, 1.327228): 3,
 (173.01448, 1.328065): 3,
 (173.01844, 1.3296509): 3,
 (173.0224, 1.3316332): 3,
 (173.02663, 1.3338798): 3,
 (173.03082, 1.3358401): 3,
 (173.03458, 1.3384391): 3,
 (173.03925, 1.3411703): 3,
 (173.04278, 1.3432848): 3,
 (173.04604, 1.3458398): 3,
 (1

In [49]:
evidence_dict

{}

In [45]:
model_location_dict['variables']['Tm_offshore'].keys()

dict_keys(['label', 'discretisation', 'child_nodes', 'training_data_preprocessed', 'testing_data_preprocessed', 'training_data', 'testing_data', 'bin_edges', 'id', 'resulting_probs'])

In [26]:
evidence_dict

{}

In [25]:
ocean_model_dict['variables'][''.keys()

dict_keys(['Tm_offshore', 'Hs_offshore', 'Dir_offshore', 'MSL', 'Tide', 'TWL', 'TWL_less_Tide', 'reef_width', 'reef_depth', 'forereef_slope', 'shore_dir', 'TWL_t_1'])

In [9]:
resulting_probs_lagoon

{0: {'VeryLow': 0.054708676330848756,
  'Low': 0.19002436468736197,
  'LowMid': 0.22277430848229593,
  'Mid': 0.27257972401775915,
  'MidHigh': 0.17688181127285996,
  'High': 0.07078355956504193,
  'VeryHigh': 0.012247555643832318}}

In [12]:
# Create an empty dictionary for the evidence and populate as you go
ocean_evidence_dict = {}

# Create a list of variables that are location specific to set as evidence in the network


# get the probability dictionary ### need to adjust so that it's not just the most likely, but the full distribution...
location_probabilities_dict = location_probabilities(ocean_evidence_dict,ocean_model_dict,variable_list,df_ocean_profiles)

# Create dataframe to plot
df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
df_twl_locations.reset_index(drop=True,inplace=True)

# data_ocean = data2geojson(df_twl_locations)

In [18]:
location_probabilities_dict

{(172.90162, 1.3804422): 3,
 (172.89642, 1.3572711): 3,
 (172.91862, 1.3429104): 3,
 (172.92461, 1.3422936): 3,
 (172.9299, 1.3429104): 3,
 (172.93386, 1.343439): 3,
 (172.93633, 1.3425579): 3,
 (172.93906, 1.3415007): 3,
 (172.9425, 1.3401792): 3,
 (172.94681, 1.3384171): 3,
 (172.95016, 1.3369194): 3,
 (172.95302, 1.3344084): 3,
 (172.95606, 1.3321618): 3,
 (172.96069, 1.3299592): 3,
 (172.96461, 1.3270518): 3,
 (172.9691, 1.3238361): 3,
 (172.97192, 1.3219419): 3,
 (172.97606, 1.3214133): 3,
 (172.97972, 1.3210168): 3,
 (172.9847, 1.3209287): 3,
 (172.98923, 1.3214573): 3,
 (172.99404, 1.3225145): 3,
 (172.99813, 1.322911): 3,
 (173.00302, 1.3241444): 3,
 (173.00703, 1.32551): 3,
 (173.00884, 1.3257743): 3,
 (173.01161, 1.327228): 3,
 (173.01448, 1.328065): 3,
 (173.01844, 1.3296509): 3,
 (173.0224, 1.3316332): 3,
 (173.02663, 1.3338798): 3,
 (173.03082, 1.3358401): 3,
 (173.03458, 1.3384391): 3,
 (173.03925, 1.3411703): 3,
 (173.04278, 1.3432848): 3,
 (173.04604, 1.3458398): 3,
 (1

In [36]:
location_probabilities_dict

{(172.90162, 1.3804422): 3,
 (172.89642, 1.3572711): 3,
 (172.91862, 1.3429104): 3,
 (172.92461, 1.3422936): 3,
 (172.9299, 1.3429104): 3,
 (172.93386, 1.343439): 3,
 (172.93633, 1.3425579): 3,
 (172.93906, 1.3415007): 3,
 (172.9425, 1.3401792): 3,
 (172.94681, 1.3384171): 3,
 (172.95016, 1.3369194): 3,
 (172.95302, 1.3344084): 3,
 (172.95606, 1.3321618): 3,
 (172.96069, 1.3299592): 3,
 (172.96461, 1.3270518): 3,
 (172.9691, 1.3238361): 3,
 (172.97192, 1.3219419): 3,
 (172.97606, 1.3214133): 3,
 (172.97972, 1.3210168): 3,
 (172.9847, 1.3209287): 3,
 (172.98923, 1.3214573): 3,
 (172.99404, 1.3225145): 3,
 (172.99813, 1.322911): 3,
 (173.00302, 1.3241444): 3,
 (173.00703, 1.32551): 3,
 (173.00884, 1.3257743): 3,
 (173.01161, 1.327228): 3,
 (173.01448, 1.328065): 3,
 (173.01844, 1.3296509): 3,
 (173.0224, 1.3316332): 3,
 (173.02663, 1.3338798): 3,
 (173.03082, 1.3358401): 3,
 (173.03458, 1.3384391): 3,
 (173.03925, 1.3411703): 3,
 (173.04278, 1.3432848): 3,
 (173.04604, 1.3458398): 3,
 (1

In [34]:
pd.DataFrame.from_dict(location_probabilities_dict,orient='index')

,0
"(172.90162, 1.3804422)",3
"(172.89642, 1.3572711)",3
"(172.91862, 1.3429104)",3
"(172.92461, 1.3422936)",3
"(172.9299, 1.3429104)",3
...,...
"(172.89827, 1.3692531)",3
"(172.90065, 1.3462583)",3
"(172.95393, 1.6455877)",3
"(173.04952, 1.3487031)",3


In [31]:
df_lagoon

,TWL,Tide,MSL,wind_u,wind_v,time,TWL_less_Tide,Hs_offshore,Tm_offshore,Dir_offshore,long,lat
0,0.208591,0.107750,0.088471,0.207214,-1.113144,1993-01-06 00:00:00.000000,0.100841,0.919050,7.219531,243.438797,172.92263,1.358684
1,0.558604,0.452697,0.088416,0.597412,-1.259857,1993-01-06 00:59:59.999997,0.105907,0.905869,7.184422,242.904663,172.92263,1.358684
2,0.814906,0.713712,0.088361,0.734024,-1.445007,1993-01-06 02:00:00.000003,0.101193,0.894031,7.136898,242.697632,172.92263,1.358684
3,0.926768,0.825621,0.088307,0.645584,-2.000870,1993-01-06 03:00:00.000000,0.101147,0.882968,7.080943,242.759918,172.92263,1.358684
4,0.859453,0.758048,0.088252,0.945312,-3.432281,1993-01-06 03:59:59.999997,0.101405,0.872668,7.023418,243.057816,172.92263,1.358684
...,...,...,...,...,...,...,...,...,...,...,...,...
3615,0.267540,-0.254680,0.087432,3.178680,0.185455,1993-01-06 18:59:59.999997,0.522221,0.213412,1.875810,258.507080,172.97082,1.568399
3616,-0.007965,-0.559890,0.087378,2.214493,0.678818,1993-01-06 20:00:00.000003,0.551925,0.192730,1.583964,258.807404,172.97082,1.568399
3617,-0.161496,-0.737260,0.087323,1.777710,0.224762,1993-01-06 21:00:00.000000,0.575764,0.173490,1.431452,259.189728,172.97082,1.568399
3618,-0.152408,-0.730377,0.087268,2.006165,-0.687057,1993-01-06 21:59:59.999997,0.577969,0.164677,1.389180,259.917908,172.97082,1.568399


# Defining times of interest


In [ ]:
# # Create an empty dictionary to have a look at how the resulting probabilties varied through time
# resulting_probs_dict = {}

# for time,model_dict['lagoon'] in model_dicts_through_time_dict.items():
    
#     #Create an empty dictionary of evidence for now
#     evidence_dict = {}
    
#     # Add evidence to model dict
#     model_location_dict = BNModel().add_evidence_to_dict(model_dict,evidence_dict)

#     # Set evidence and get beliefs
#     model_location_dict = BNModel().update_evidence(model_location_dict)
    
#     # Get the resulting TWL probabilities (right now for a generic location)
#     resulting_probs = model_location_dict['variables']['TWL']['resulting_probs']
    
#     # Add the resulting probabilities to a dictionary
#     resulting_probs_dict.update({
#         time:resulting_probs[0]
#     })

In [ ]:
# resulting_probs_dict

# Interactive Plots

In [ ]:
# # load in the reef and shoreline profile information
# df_ocean_profiles = pd.read_csv('/src/Dataset/D8_tarawa_inundation/Profiles_definition_outer_reef_xyxy_processed.txt')
# df_lagoon_profiles = pd.read_csv('/src/Dataset/D8_tarawa_inundation/Profiles_definition_inner_lagoon_xyxy.txt',delim_whitespace=True,header=None)
# df_lagoon_profiles.columns = ['reef_long','reef_lat','shore_long','shore_lat','reef_depth']


In [ ]:
# def bin_locator(value,bin_edges):
#     '''
#     function used for determining the index of the appropriate bin for a numerical value.
#     '''
#     i=0
#     for edge_1,edge_2 in zip(bin_edges[:-1],bin_edges[1:]):
#         if (value>edge_1)&(value<=edge_2):
#             loc_bin = i
#         else:
#             i+=1
#             continue

#     if value<=bin_edges[0]:
#         loc_bin = 0

#     if value>=bin_edges[-1]:
#         loc_bin = len(bin_edges)-2

#     return(loc_bin)

# def model_location(model_dict,location_details,evidence_dict,variable_list):
    
#     '''
#     function for adding the location information for one side model to the evidence dictionary
#     '''
    
#     for variable in variable_list:

#         bin_edges = model_dict['variables'][variable]['bin_edges'][0]
#         value = location_details[variable]

#         var_bin = bin_locator(value,bin_edges)
        
#         evidence_array = [0]*(len(bin_edges)-1)
#         evidence_array[var_bin] = 1
        
#         evidence_dict.update({
#             variable:evidence_array
#         })

#     # Add evidence to model dict
#     model_location_dict = BNModel().add_evidence_to_dict(model_dict,evidence_dict)
    
#     # Set evidence and get beliefs
#     model_location_dict = BNModel().update_evidence(model_location_dict)
    
#     return(model_location_dict)

# def location_probabilities(evidence_dict,model_dict,variable_list,df_profiles):
#     '''
    
#     Function for setting evidence and determing probabilties for twl at each point around the island based 
#     on the reef characteristics at each location
    
#     '''
    
#     location_probabilities_dict = {}
# #     figure_dict = {}
    
#     for index,row in df_profiles.iterrows():

#         model_location_dict = model_location(model_dict,row,evidence_dict,variable_list)
#         location_probabilities = model_location_dict['variables']['TWL']['resulting_probs'][0]

#         df_location_probabilities = pd.DataFrame.from_dict(location_probabilities,orient='index')
        
# #         # Create figure for popup
# #         fig = plt.figure(figsize=(2,2))
# #         plt.bar(x=df_location_probabilities.index,height=df_location_probabilities[0])
# #         plt.savefig('{}_{}.png'.format(int(row.reef_long*1000),int(row.reef_lat*1000)))
# #         plt.close()
        
#         largest_cat = df_location_probabilities.idxmax()[0]

#         location_probabilities_dict.update({
#             (row.reef_long,row.reef_lat):\
#                 model_dict['variables']['TWL']['discretisation']['bin_names'].index(largest_cat)
#         })
        
# #         figure_dict.update({
# #             (row.reef_long,row.reef_lat):fig#html_graph
# #         })
        
#     return(location_probabilities_dict)

# def data2geojson(df):
#     features = []
#     insert_features = lambda X: features.append(
#             geojson.Feature(geometry=geojson.Point((X["long"],
#                                                     X["lat"])),
#                             properties=dict(name=X["most_likely_twl"])))
#     df.apply(insert_features, axis=1)
        
#     return(geojson.FeatureCollection(features))

# # Load SLR Projections
# data_location = "/src/Dataset/D7_MSL_projections/"
# file_name = "distributions_dict"
# with open("{}{}.json".format(data_location,file_name), 'r') as fp:
#     SL_proj_dict = json.load(fp)
    
# def SLR_proj_extractor(SL_proj_dict,AIS_config,rcp,year):
#     '''
#     Function for getting SLR projections for a given Antarctic icesheet, rcp and year
#     Years start as 2020 and go up in lots of 10 until 2150 (2100 for dp16)
#     '''
#     SLR_prob_dict = SL_proj_dict["('{}', '{}', {})".format(AIS_config,rcp,year)]
#     SLR_median_prob = np.max([float(x) for x in list(SLR_prob_dict.keys())])
#     SLR_median_MSL = float(SLR_prob_dict[str(SLR_median_prob)])/1000 #units is m

#     return(SLR_median_MSL)

In [ ]:
# def test_figure(
#     view,tide_bin,wave_height_bin,wave_period_bin,wave_direction_bin,wind_u_bin,wind_v_bin,proj_time,time
#     ):
#     # Extract the right model from the dictionary of model
#     lagoon_model_dict = model_dicts_through_time_dict[time]['lagoon']
#     ocean_model_dict = model_dicts_through_time_dict[time]['ocean']
    
#     ########### get the MSL bin based on slider value
#     msl_proj = SLR_proj_extractor(SL_proj_dict,'k14','26','{}'.format(proj_time))
# #     ocean_model_dict['variables']['MSL']['bin_edges'][0]
#     bin_count = bin_locator(msl_proj,ocean_model_dict['variables']['MSL']['bin_edges'][0])
#     msl_bin = ocean_model_dict['variables']['MSL']['discretisation']['bin_names'][bin_count]
    
#     if view == 'Map':
#         map_osm = folium.Map(location=[1.448888, 172.991794],zoom_start=11)
#     elif view == 'Satellite':
#         token = "pk.eyJ1Ijoic2hhbm5vbi1iZW5ndHNvbiIsImEiOiJja3F1Y2Q0dHEwMzYwMm9wYmtzYzk2bDZuIn0.5jGMyEiJdmXs1HL7x3ThPw" # your mapbox token
#         tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

#         map_osm = folium.Map(location=[1.448888, 172.991794], zoom_start=11, tiles=tileurl, attr='Mapbox')
        
#     twl_bin_edges = [round(x,2) for x in ocean_model_dict['variables']['TWL']['bin_edges'][0]]
#     twl_bins = ocean_model_dict['variables']['TWL']['discretisation']['bin_names']

#     colours_rgb = matplotlib.pyplot.get_cmap('seismic')(np.arange(0,1+1/len(twl_bins),1/(len(twl_bins)-1)))
#     colour_hex_dict = {i:rgb2hex(int(255*colours_rgb[i][0]),int(255*colours_rgb[i][1]),int(255*(colours_rgb[i][2]))) for i in np.arange(0,len(twl_bins),1)}
    
#     ################################################
    
#     # Create an empty dictionary for the evidence and populate as you go
#     ocean_evidence_dict = {}
    
#     for var_bin,var_name in zip([tide_bin,msl_bin,wave_height_bin,wave_period_bin,wave_direction_bin],
#                                 ['Tide','MSL','Hs_offshore','Tm_offshore','Dir_offshore']):

#         ## Set in the evidence dict to be as indicated in the dropdown
#         bin_index = ocean_model_dict['variables'][var_name]['discretisation']['bin_names'].index(var_bin)
#         # Create a list of the tide evidence (all zero except as indicated by dropdown. Dropdown=1)
#         evidence = [0 for x in ocean_model_dict['variables'][var_name]['discretisation']['bin_names']]
#         evidence[bin_index] = 1
#         ocean_evidence_dict.update({
#             var_name:evidence
#         })
    
#     # Create a list of variables that are location specific to set as evidence in the network
#     variable_list = ['reef_width','reef_depth','forereef_slope','shore_dir']
    
#     # get the probability dictionary
#     location_probabilities_dict = location_probabilities(ocean_evidence_dict,ocean_model_dict,variable_list,df_ocean_profiles)
    
#     # Create dataframe to plot
#     df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
#     df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
#     df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
#     df_twl_locations.reset_index(drop=True,inplace=True)
    
#     data_ocean = data2geojson(df_twl_locations)
    
#     colors_hex_points_ocean = [colour_hex_dict[x] for x in df_twl_locations.most_likely_twl]
    
#     #####################################################################
    
#     # Create an empty dictionary for the evidence and populate as you go
#     lagoon_evidence_dict = {}
    
#     for var_bin,var_name in zip([tide_bin,msl_bin,wave_height_bin,wave_period_bin,wave_direction_bin,wind_u_bin,wind_v_bin],
#                                 ['Tide','MSL','Hs_offshore','Tm_offshore','Dir_offshore','wind_u','wind_v']):

#         ## Set in the evidence dict to be as indicated in the dropdown
#         bin_index = lagoon_model_dict['variables'][var_name]['discretisation']['bin_names'].index(var_bin)
#         # Create a list of the tide evidence (all zero except as indicated by dropdown. Dropdown=1)
#         evidence = [0 for x in lagoon_model_dict['variables'][var_name]['discretisation']['bin_names']]
#         evidence[bin_index] = 1
#         lagoon_evidence_dict.update({
#             var_name:evidence
#         })
    
#     # Create a list of variables that are location specific to set as evidence in the network
#     variable_list = []
    
#     # get the probability dictionary
#     location_probabilities_dict = location_probabilities(lagoon_evidence_dict,lagoon_model_dict,variable_list,df_lagoon_profiles)
    
#     # Create dataframe to plot
#     df_twl_locations = pd.DataFrame.from_dict(location_probabilities_dict,orient='index').rename(columns={0:'most_likely_twl'})
#     df_twl_locations['long'] = [long for long,lat in df_twl_locations.index]
#     df_twl_locations['lat'] = [lat for long,lat in df_twl_locations.index]
#     df_twl_locations.reset_index(drop=True,inplace=True)
    
#     data_lagoon = data2geojson(df_twl_locations)
    
#     colors_hex_points_lagoon = [colour_hex_dict[x] for x in df_twl_locations.most_likely_twl]
    
#     #####################################################################
    
#     features_list = data_ocean['features']+data_lagoon['features']
    
#     data = data_ocean
#     data.update({
#         'features':features_list
#     })
    
#     colors_hex_points = colors_hex_points_ocean+colors_hex_points_lagoon
    
#     #####################################################################

#     for feature,color in zip(features_list,colors_hex_points):
#         feature['properties'] = {'color':color,'weight':1,'markerColor':color,'fillOpacity':1,'fillColor':color}
#         long,lat = feature['geometry']['coordinates']
        
#         marker = folium.CircleMarker([lat,long],color=color,
#                                     # popup='<img src={}_{}.png>'.format(int(long*1000),int(lat*1000)),
#                                    fill_color=color,fill=True,fill_opacity='1',radius=5)
#         marker.add_to(map_osm)
        
#     twl_bin_edge_labels = ['{} to {} m'.format(
#         x,y) for x,y in zip(twl_bin_edges[:-1],twl_bin_edges[1:])]
        
#     output_list = []
#     for rgb_color in colours_rgb:
#         output = plt.scatter([],[],color=rgb_color)
#         output_list.append(output)
        
#     legend = plt.legend(output_list,twl_bin_edge_labels,title='Total water level anomaly',fontsize=10)
#     plt.setp(legend.get_title(),fontsize=12)
    
#     plt.axis('off')
#     plt.savefig('legend.png')
    
#     plt.close()
    
#     url = (
#         "legend.png"
#     )    
    
#     FloatImage(url, bottom=55, left=55).add_to(map_osm)
    
#     map_osm.save('test.html')
        
#     return(map_osm)

# # compile the figure
# lagoon_model_dict,ocean_model_dict = initialise_model_dictionaries()
# tide_bins = ocean_model_dict['variables']['Tide']['discretisation']['bin_names']
# wave_height_bins = ocean_model_dict['variables']['Hs_offshore']['discretisation']['bin_names']
# wave_period_bin = ocean_model_dict['variables']['Tm_offshore']['discretisation']['bin_names']
# wave_direction_bin = ocean_model_dict['variables']['Dir_offshore']['discretisation']['bin_names']
# wind_u_bin = lagoon_model_dict['variables']['wind_u']['discretisation']['bin_names']
# wind_v_bin = lagoon_model_dict['variables']['wind_u']['discretisation']['bin_names']
# time = list(model_dicts_through_time_dict.keys())  

# # Create the plot with the widget
# map_osm = interact_manual(test_figure,
#                 view = widgets.Dropdown(options=['Map','Satellite'],value='Map',description='View type',disabled=False),
#                 tide_bin = widgets.Dropdown(options=tide_bins,value='Mid',description='Tide',disabled=False),
#                 wave_height_bin = widgets.Dropdown(options=wave_height_bins,value='Mid',description='Wave height',disabled=False),
#                 wave_period_bin = widgets.Dropdown(options=wave_period_bin,value='Mid',description='Wave period',disabled=False),
#                 wave_direction_bin = widgets.Dropdown(options=wave_direction_bin,value='NNE',description='Wave direction',disabled=False),
#                 wind_u_bin = widgets.Dropdown(options=wind_u_bin,value='Mid',description='Wind u',disabled=False),
#                 wind_v_bin = widgets.Dropdown(options=wind_v_bin,value='Mid',description='Wind v',disabled=False),
#                 proj_time = widgets.IntSlider(min=2020,max=2150,step=10,value=2020,description='SLR prediction'),
#                 time = widgets.Dropdown(options=time,value=727930.0,description='Time',disabled=False)
#                )

# map_osm